In [1]:
# 加载 TensorBoard notebook 插件
%load_ext tensorboard

In [2]:
# @RefLink: https://www.tensorflow.org/tensorboard/scalars_and_keras?hl=zh-cn
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version
import numpy as np

import tensorflow as tf
from tensorflow import keras

from utils.dir_utils import makedir_exist_ok

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.3.0


In [3]:
data_size = 1000
# 80% 的数据用来训练
train_pct = 0.8

train_size = int(data_size * train_pct)

# 创建在(-1,1)范围内的随机数作为输入
x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

# 生成输出数据
# y = 0.5x + 2 + noise
y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

# 将数据分成训练和测试集
x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

## Logging custom scalars

In [4]:
log_dir = "./logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
makedir_exist_ok(log_dir)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

# Logging custom scalars
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()

def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  learning_rate = 0.2
  if epoch > 10:
    learning_rate = 0.02
  if epoch > 20:
    learning_rate = 0.01
  if epoch > 50:
    learning_rate = 0.005
  # see https://www.tensorflow.org/tensorboard/scalars_and_keras
  # Inside the learning rate function, use tf.summary.scalar() to log the custom learning rate.
  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

model = keras.models.Sequential([
    keras.layers.Dense(16, input_dim=1),
    keras.layers.Dense(1),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=keras.optimizers.SGD(),
)

print("Training ... With default parameters, this takes less than 10 seconds.")
training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=train_size,
    verbose=0, # Suppress chatty output; use Tensorboard instead
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback, lr_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))

Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Training ... With default parameters, this takes less than 10 seconds.
Average test loss:  0.0026170198270119727


## 记录自定义 scalars

In [5]:
%tensorboard --logdir logs/scalars

Reusing TensorBoard on port 6006 (pid 12224), started 6:36:45 ago. (Use '!kill 12224' to kill it.)